In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('song.csv')

In [3]:
df

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \nLet the angels fly l...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \nMore power \nPower to...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \nis something i'll believe \nf...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \nam i frightened \nwhere can ...


In [4]:
df.shape

(57650, 4)

In [5]:
df=df.sample(n=5000).drop('link',axis=1).reset_index(drop=True)

In [6]:
df

,artist,song,text
0,Procol Harum,Your Own Choice,There's too many women and not enough wine \n...
1,Pet Shop Boys,If Love Were All,I believe in doing what I can \nIn crying whe...
2,Gloria Estefan,Higher,"Here we go, here we go now \n \nI saw a girl..."
3,Waylon Jennings,Alone,Isn't there anyone \nWho'll take me like I am...
4,Westlife,Fragile Heart,"A fragile heart, was broken before, \nI don't..."
...,...,...,...
4995,Adam Sandler,Grow Old With You,I wanna make you smile whenever you're sad \n...
4996,Wu-Tang Clan,Shame On A Nuh,"Yo, this the O.D.B. \nNew radio version for y..."
4997,Robbie Williams,South Of The Border,I know a freaky young lady \nName of Cocaine ...
4998,John Denver,Gimme Your Love,"Oh, spring is here and love is in the air \nO..."


# Cleaning

In [7]:
df['song'][0]

'Your Own Choice'

In [8]:
df['text']=df['text'].str.lower().replace(r'[^\w\s]','').replace('\\n','', regex= True)

In [9]:
df['text'][0]

"there's too many women and not enough wine  too many poets and not enough rhyme  too many glasses and not enough time  draw your own conclusions    my old dog's a good old dog  my old man's a silly old sod  the human face is a terrible place  choose your own examples    went to the river, but i could not swim  knew i'd drown if i went in  lost my faith in a terrible race  rest-in-peace hereafter"

In [10]:
import nltk
from nltk.stem.porter import PorterStemmer

ps= PorterStemmer()


def tokenization(txt):
    tokens= nltk.word_tokenize(txt)
    stemming= [ps.stem(i) for i in tokens]
        
    return " " . join(stemming)

In [11]:
tokenization('this is muy loving loved someonne')

'thi is muy love love someonn'

In [12]:
df['text']=df['text'].apply(lambda x: tokenization(x))

In [13]:
df['text']

0       there 's too mani women and not enough wine to...
1       i believ in do what i can in cri when i must i...
2       here we go , here we go now i saw a girl today...
3       is n't there anyon who 'll take me like i am ?...
4       a fragil heart , wa broken befor , i do n't th...
                              ...                        
4995    i wan na make you smile whenev you 're sad car...
4996    yo , thi the o.d.b . new radio version for ya ...
4997    i know a freaki young ladi name of cocain kati...
4998    oh , spring is here and love is in the air ove...
4999    we are young , we run free stay up late , we d...
Name: text, Length: 5000, dtype: object

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
tfid= TfidfVectorizer(stop_words='english')
matrix= tfid.fit_transform(df['text'])

In [16]:
matrix.shape

(5000, 17711)

In [17]:
sim= cosine_similarity(matrix)

In [18]:
sim[0]

array([1.        , 0.03544953, 0.00104898, ..., 0.05344926, 0.        ,
       0.00117057])

In [19]:
df['song'][0]

'Your Own Choice'

In [27]:
df[df['song']== 'Party In The CIA']

,artist,song,text


In [21]:
distances =sorted(list(enumerate (sim[0])),reverse= False, key=lambda x:x[1])

In [22]:
def recommendation(song):
    idx=df[df['song']==song].index[0]
    distances =sorted(list(enumerate (sim[idx])),reverse= False, key=lambda x:x[1])
    
    songs=[]
    
    for i in distances[1:21]:
        songs.append(df.iloc[i[0]].song)
    return songs

In [23]:
recommendation('La Vie En Rose')

['War Games',
 'May This Be Love',
 'Christian Brothers',
 "Lt's Midnight Dream",
 'The Sound Of The Crowd',
 'Oh The Guilt',
 'Big Socks',
 'Islands',
 'Softly And Tenderly',
 'Sweet Dream',
 '89 Vision',
 "It Ain't Necessarily So",
 'Fujiyama Mama',
 'Never Know Why',
 'The Connection',
 'For Calvin',
 'Cool Water',
 'Sa Ugoy Ng Duyan',
 "My Bucket's Got A Hole In It",
 'Ada Lagi Yang Mati']

In [24]:
import pickle

In [25]:
pickle.dump(df, open('df.pkl', 'wb'))

In [26]:
pickle.dump(sim, open('sim.pkl','wb'))